In [1]:
import numpy as np 
import pandas as pd 

In [3]:
combine_file = '/Users/gengyunxin/Documents/项目/traffic_model/data/California/com_E_work_and_week_all.csv'
df = pd.read_csv(combine_file).transpose() # 13*24*61天=17568点
df.head()

,0,1,2,3,4,5,6,7,8,9,...,17558,17559,17560,17561,17562,17563,17564,17565,17566,17567
767838,143.0,194.0,182.0,141.0,148.0,160.0,143.0,175.0,203.0,157.0,...,186.0,214.0,191.0,159.0,173.0,179.0,198.0,185.0,173.0,177.0
773656,138.0,198.0,196.0,158.0,108.0,189.0,147.0,158.0,189.0,162.0,...,207.0,211.0,186.0,177.0,178.0,197.0,191.0,165.0,188.0,172.0
760074,177.0,200.0,253.0,211.0,193.0,229.0,180.0,215.0,215.0,160.0,...,260.0,265.0,250.0,226.0,235.0,275.0,256.0,216.0,252.0,230.0
760080,199.0,248.0,296.0,235.0,216.0,265.0,202.0,206.0,240.0,200.0,...,358.0,323.0,288.0,258.0,261.0,232.0,272.0,278.0,242.0,227.0
716414,255.0,266.0,319.0,278.0,234.0,317.0,251.0,267.0,271.0,216.0,...,82.0,80.0,72.0,67.0,66.0,66.0,65.0,63.0,59.0,53.0


In [7]:
# 早：5:00 ～ 12:55 （8小时、96点） [0]
# 中：13:00 ～ 20:55 （8小时、96点） [1]
# 晚：21:00 ～ 4:55 （8小时、96点）[2]
df1 = df.iloc[:, 60:-36]
df1.head()

,60,61,62,63,64,65,66,67,68,69,...,17522,17523,17524,17525,17526,17527,17528,17529,17530,17531
767838,117.0,93.0,98.0,109.0,108.0,97.0,111.0,125.0,122.0,120.0,...,225.0,221.0,221.0,187.0,166.0,220.0,210.0,203.0,204.0,179.0
773656,115.0,102.0,101.0,120.0,142.0,123.0,139.0,166.0,161.0,152.0,...,272.0,265.0,267.0,274.0,214.0,247.0,261.0,229.0,213.0,183.0
760074,154.0,136.0,138.0,150.0,176.0,166.0,199.0,211.0,237.0,231.0,...,403.0,380.0,402.0,374.0,306.0,347.0,405.0,340.0,340.0,323.0
760080,154.0,139.0,146.0,173.0,188.0,154.0,215.0,223.0,252.0,247.0,...,275.0,290.0,276.0,276.0,248.0,272.0,334.0,252.0,255.0,236.0
716414,217.0,200.0,210.0,236.0,242.0,274.0,284.0,320.0,319.0,318.0,...,445.0,456.0,461.0,396.0,390.0,422.0,416.0,370.0,420.0,390.0


In [12]:
df1.shape

(24, 17472)

In [20]:
df1.iloc[0]

60       117.0
61        93.0
62        98.0
63       109.0
64       108.0
         ...  
17527    220.0
17528    210.0
17529    203.0
17530    204.0
17531    179.0
Name: 767838, Length: 17472, dtype: float64

In [19]:
num_roads = 24
# 每条路的数据独立做归一化
raw = []
for i in range(num_roads):
    array = df1.iloc[i]
    max = array.max()
    min = array.min()
    normalized = ((array - min) / (max - min)).tolist()
    raw.append(normalized)

df_norm = np.array(raw)
    
data_onethird = df_norm.reshape(-1, 96) # 每条样本96个点,即1/3天 
data_onethird

array([[0.28715365, 0.22670025, 0.23929471, ..., 0.62720403, 0.60201511,
        0.52896725],
       [0.53400504, 0.70277078, 0.72292191, ..., 0.45088161, 0.49118388,
        0.5768262 ],
       [0.51889169, 0.47607053, 0.55415617, ..., 0.23677582, 0.19647355,
        0.18891688],
       ...,
       [0.39444444, 0.39722222, 0.39722222, ..., 0.18333333, 0.19444444,
        0.2       ],
       [0.21388889, 0.25555556, 0.3       , ..., 0.68611111, 0.68055556,
        0.67777778],
       [0.66944444, 0.66944444, 0.68333333, ..., 0.40277778, 0.35      ,
        0.34722222]])

In [21]:
data_onethird.shape

(4368, 96)

In [24]:
l = ([0, 1, 2] * 60 + [0,1]) * num_roads
labels = np.array(l)
len(labels)

4368

In [25]:
num_rows = data_onethird.shape[0]
data_with_label = np.insert(data_onethird, 96, values=labels, axis=1) # (4368, 97)
data_with_label

array([[0.28715365, 0.22670025, 0.23929471, ..., 0.60201511, 0.52896725,
        0.        ],
       [0.53400504, 0.70277078, 0.72292191, ..., 0.49118388, 0.5768262 ,
        1.        ],
       [0.51889169, 0.47607053, 0.55415617, ..., 0.19647355, 0.18891688,
        2.        ],
       ...,
       [0.39444444, 0.39722222, 0.39722222, ..., 0.19444444, 0.2       ,
        2.        ],
       [0.21388889, 0.25555556, 0.3       , ..., 0.68055556, 0.67777778,
        0.        ],
       [0.66944444, 0.66944444, 0.68333333, ..., 0.35      , 0.34722222,
        1.        ]])

In [26]:
df = pd.DataFrame(data_with_label)
df.to_csv('/Users/gengyunxin/Documents/项目/traffic_model/test/data/processed_finetune_seattle_96.csv', index=0, header=0)